# **User Feature Metric Overview**

In [27]:
import os  # type: ignore
import sys  # type: ignore
import pandas as pd # type: ignore
import numpy as np  # type: ignore
import math
from sklearn.preprocessing import OneHotEncoder # type: ignore
# ============================================================
# :weißes_häkchen: Add the project root to sys.path (not /core)
# ============================================================
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)
print("Project root added to sys.path:", project_root)

Project root added to sys.path: /Users/claudiatagbo/Masterschool/traveltide


In [28]:
# Verify that core is importable
try:
    import core
    print(":weißes_häkchen: core module found at:", core.__file__)
except ModuleNotFoundError:
    print(":x: core module not found. Check your sys.path!")


:weißes_häkchen: core module found at: None


## **Speicherpfad für Feature-Metriken**
**

In [29]:
# Pfad für feature_Metric-Abbildungen
feature_metrics_path = os.path.join(project_root, 'data', 'processed', 'feature_metrics')
#dashboard_path = os.path.join(eda_figures_path, 'dashboard')
#eda_data_path = os.path.join(project_root, 'reports', 'eda', 'results')
os.makedirs(feature_metrics_path, exist_ok=True)
#os.makedirs(dashboard_path, exist_ok=True)
#os.makedirs(eda_data_path, exist_ok=True)

## **Mittels Methode**

In [30]:
def haversine(lat1, lon1, lat2, lon2):
        """Calculate distance between two coordinates using Haversine formula."""
        #Haversine-Formel bestimmt die Entfernung zwischen zwei Punkten auf der Erde in Kilometern
        R = 6371  # Earth radius in kilometers
        try:
            lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            return R * c
        except (ValueError, TypeError):
            return np.nan
def get_age(row):
        """Calculate age from birthdate."""
        #Berechnet das Alter (in Jahren) aus einem Geburtsdatum, das in einer Tabellenzeile (row['birthdate']) gespeichert ist.
        try:
            birth_date = pd.to_datetime(row['birthdate'], format='mixed', errors='coerce')
            if pd.isnull(birth_date):
                return np.nan
            return (pd.Timestamp.now().normalize() - birth_date).days / 365
        except:  # noqa: E722
            return np.nan
def is_group_trip(row):
        """Group trip: multiple seats and rooms."""
        return int(
            row.get('flight_booked', False) and
            row.get('return_flight_booked', False) and
            row.get('hotel_booked', False) and
            row.get('seats', 0) > 2 and
            row.get('rooms', 0) > 1
        )
def is_pair_trip(row):
        """Couple trip: exactly 2 seats and 1 room."""
        return int(
            row.get('flight_booked', False) and
            row.get('return_flight_booked', False) and
            row.get('hotel_booked', False) and
            row.get('seats', 0) == 2 and
            row.get('rooms', 0) == 1
        )
def is_business_week_trip(row):
        """Business trip: weekday travel, short stays, age 25–60."""
        age = get_age(row)
        departure = row.get('departure_time')
        return_ = row.get('return_time')
        if pd.isnull(departure) or pd.isnull(return_):
            return 0
        return int(
            row.get('flight_booked', False) and
            row.get('return_flight_booked', False) and
            row.get('hotel_booked', False) and
            row.get('seats', 0) == 1 and
            row.get('nights', 0) >= 1 and
            row.get('nights', 0) < 6 and
            25 <= age <= 60 and
            departure.weekday() <= 4 and
            return_.weekday() <= 4
        )
def is_weekend_trip_new(row):
        """Weekend trip: ≤2 nights, Fri–Sun travel."""
        departure = row.get('departure_time')
        return_ = row.get('return_time')
        if pd.isnull(departure) or pd.isnull(return_):
            return 0
        return int(
            row.get('flight_booked', False) and
            row.get('return_flight_booked', False) and
            row.get('hotel_booked', False) and
            row.get('nights', 0) <= 2 and
            departure.weekday() >= 4 and
            return_.weekday() <= 6
        )
def is_discount_trip_new(row):
    #Bestimmt, ob eine Reise mit Rabatt gebucht wurde — egal, ob der Rabatt für den Flug oder das Hotel gilt.
        """Trip with any discount applied."""
        return int(row.get('hotel_discount', False) or row.get('flight_discount', False))
def get_season(row):
    month = row['departure_time'].month
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

## ** Laden Users und Session**

In [31]:
from core.load_data import load_table  # type: ignore

In [32]:
users = load_table(data_type='raw', table_name='users')
sessions = load_table(data_type='processed', table_name='sessions_cleaned')
nc_sessions = load_table(data_type='processed', table_name='sessions_not_canceled_trips')

 Lade Tabelle 'users' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/raw/users.csv
 CSV geladen. Zeilen: 1020926
 Lade Tabelle 'sessions_cleaned' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/processed/sessions_cleaned.csv
 CSV geladen. Zeilen: 49211
 Lade Tabelle 'sessions_not_canceled_trips' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/processed/sessions_not_canceled_trips.csv
 CSV geladen. Zeilen: 16099


## **Koventierung der Relevante Datumsspalten**

In [33]:
def preprocess_dates():
    """
    Konvertiert relevante Datumsspalten in datetime-Objekte.
    """
    # ----------------------------------------------------
    # 1. Verarbeitung der Daten für den vollständigen Sessions-DataFrame (self.df_sessions)
    # ----------------------------------------------------
    print("--- Vorverarbeitung von self.df_sessions (Alle Sessions) wird gestartet ---")
    sessions['session_start'] = pd.to_datetime(sessions['session_start'])
    print("Spalte 'session_start' erfolgreich in datetime konvertiert.")
    sessions['session_end'] = pd.to_datetime(sessions['session_end'], format='mixed')
    print("Spalte 'session_end' erfolgreich in datetime konvertiert (Format='mixed').")
    sessions['departure_time'] = pd.to_datetime(sessions['departure_time'], format='mixed')
    print("Spalte 'departure_time' erfolgreich in datetime konvertiert (Format='mixed').")
    # ----------------------------------------------------
    # 2. Verarbeitung der Daten für den DataFrame der Nicht-Stornierungen (self.df_nc_sessions)
    # ----------------------------------------------------
    print("\n--- Vorverarbeitung von self.nc_sessions (Nicht stornierte Sessions) wird gestartet ---")
    nc_sessions['session_end'] = pd.to_datetime(nc_sessions['session_end'], format='mixed')
    print("Spalte 'session_end' erfolgreich in datetime konvertiert (Format='mixed').")
    nc_sessions['departure_time'] = pd.to_datetime(nc_sessions['departure_time'])
    print("Spalte 'departure_time' erfolgreich in datetime konvertiert (Format='mixed').")
    nc_sessions['return_time'] = pd.to_datetime(nc_sessions['return_time'], format='mixed')
    print("Spalte 'return_time' erfolgreich in datetime konvertiert (Format='mixed').")
    nc_sessions['check_in_time'] = pd.to_datetime(nc_sessions['check_in_time'], format='mixed')
    print("Spalte 'check_in_time' erfolgreich in datetime konvertiert (Format='mixed').")
    print("\n:weißes_häkchen: Alle Datumsspalten wurden erfolgreich in das datetime-Format konvertiert.")

In [34]:
preprocess_dates()

--- Vorverarbeitung von self.df_sessions (Alle Sessions) wird gestartet ---
Spalte 'session_start' erfolgreich in datetime konvertiert.
Spalte 'session_end' erfolgreich in datetime konvertiert (Format='mixed').
Spalte 'departure_time' erfolgreich in datetime konvertiert (Format='mixed').

--- Vorverarbeitung von self.nc_sessions (Nicht stornierte Sessions) wird gestartet ---
Spalte 'session_end' erfolgreich in datetime konvertiert (Format='mixed').
Spalte 'departure_time' erfolgreich in datetime konvertiert (Format='mixed').
Spalte 'return_time' erfolgreich in datetime konvertiert (Format='mixed').
Spalte 'check_in_time' erfolgreich in datetime konvertiert (Format='mixed').

:weißes_häkchen: Alle Datumsspalten wurden erfolgreich in das datetime-Format konvertiert.


In [35]:
def enrich_sessions():
    """
    Fügt den Session-DataFrames neue Features hinzu (z. B. Stornierungsstatus, Dauer).
    """
    print("--- Start der Session-Anreicherung (Feature Engineering) ---")
    # 1. Nicht stornierte Trip-IDs abrufen
    nc_trip_ids = nc_sessions['trip_id'].unique()
    print("Einzigartige IDs der nicht stornierten Reisen wurden abgerufen.")
    # 2. Feature: Window Shopping (Sessions ohne Trip-ID)
    sessions['window_shopping'] = sessions['trip_id'].isna().astype(int)
    print("Berechne das Feature 'window_shopping' (Session ohne Buchung).")
    # 3. Feature: Stornierte Reisen (Trip-ID existiert, ist aber nicht in nc_trip_ids enthalten)
    sessions['canceled_trip'] = sessions['trip_id'].apply(
        lambda x: 0 if pd.isna(x) else int(x not in nc_trip_ids)
    )
    print("Berechne das Feature 'canceled_trip' (Session führte zu stornierter Reise).")
  
    print("\n:weißes_häkchen: Session-DataFrames wurden erfolgreich mit neuen Features angereichert.")

## ** Feature Engineering for Sessiondaten**

In [36]:
nc_trip_ids = nc_sessions['trip_id'].unique()
print('Anzahl der bezahlten Reisen', len(nc_trip_ids))
def window_shopping(row):
  trip_id = row['trip_id']
  if pd.isna(trip_id):
    return 1
  else:
    return 0
def canceled_trip(row):
  if pd.isna(row['trip_id']):
    return 0
  else:
    return int(row['trip_id'] not in nc_trip_ids)


Anzahl der bezahlten Reisen 16099


In [37]:
def enhance_session_data():
    """
    Fügt den Session-DataFrames neue Features hinzu (z. B. Stornierungsstatus, Dauer).
    """
    print("--- Start der Session-Anreicherung (Feature Engineering) ---")
    # 1. Feature: Window Shopping (Sessions ohne Trip-ID)
    sessions['window_shopping'] = sessions.apply(window_shopping, axis = 1)
    print("Berechne das Feature 'window_shopping' (Session ohne Buchung).")
    # 2. Feature: Stornierte Reisen (Trip-ID existiert, ist aber nicht in nc_trip_ids enthalten)
    sessions['canceled_trip'] = sessions.apply(canceled_trip, axis = 1)
    print("Berechne das Feature 'canceled_trip' (Session führte zu stornierter Reise).")
    print("\n:weißes_häkchen: Session-DataFrames wurden erfolgreich mit neuen Features angereichert.")









In [38]:
enhance_session_data()

--- Start der Session-Anreicherung (Feature Engineering) ---
Berechne das Feature 'window_shopping' (Session ohne Buchung).
Berechne das Feature 'canceled_trip' (Session führte zu stornierter Reise).

:weißes_häkchen: Session-DataFrames wurden erfolgreich mit neuen Features angereichert.


In [39]:
def aggregate_user_sessions():
        return sessions.groupby('user_id').agg(
            num_clicks=('page_clicks', 'sum'),
            avg_session_clicks=('page_clicks', 'mean'),
            #max_session_clicks=('page_clicks', 'max'),
            num_empty_sessions=('window_shopping', 'sum'),
            num_canceled_trips=('canceled_trip', 'sum'),
            num_sessions=('session_id', 'nunique'),
            avg_session_duration=('session_duration', 'mean')
        ).reset_index()

In [40]:
user_base= aggregate_user_sessions()

In [41]:
user_base.columns

Index(['user_id', 'num_clicks', 'avg_session_clicks', 'num_empty_sessions',
       'num_canceled_trips', 'num_sessions', 'avg_session_duration'],
      dtype='object')

## **Wechsel zum DataFrame der nicht stornierten Reisen**


In [42]:
def filter_non_cancelled_trips():
    # ============================================================
    # 1️⃣ Datengrundlage: Nicht stornierte Reisen (nc_sessions)
    # ============================================================
    df = nc_sessions  # Arbeite mit dem DataFrame aller gültigen, nicht stornierten Sessions

    # ============================================================
    # 2️⃣ Anzahl gebuchter Flüge berechnen
    # ============================================================
    # - Wenn sowohl Hin- als auch Rückflug gebucht: 2 Flüge
    # - Wenn nur ein Flug gebucht: 1 Flug
    # - Sonst: 0 Flüge
    df['num_flights'] = np.where(
        (df['flight_booked']) & (df['return_flight_booked']), 2,
        np.where(df['flight_booked'], 1, 0)
    )

    # ============================================================
    # 3️⃣ Anzahl gebuchter Hotels bestimmen
    # ============================================================
    # - Wandelt die boolesche Spalte 'hotel_booked' in 0/1 um
    df['num_hotels'] = df['hotel_booked'].astype(int)

    # ============================================================
    # 4️⃣ Geldbetrag pro Flug berechnen
    # ============================================================
    # - Wenn ein Flugrabatt aktiv ist, wird der Rabatt auf den Basispreis angewendet
    # - Ansonsten wird der volle Preis genutzt
    df['money_spent_per_flight'] = np.where(
        df['flight_discount'],
        df['base_fare_usd'] * (1 - df['flight_discount_amount']),
        df['base_fare_usd']
    )

    # ============================================================
    # 5️⃣ Kosten pro Sitzplatz ermitteln
    # ============================================================
    # - Gesamtkosten des Flugs werden durch Anzahl der Sitze geteilt
    df['money_spent_per_seat'] = df['money_spent_per_flight'] / df['seats']

    # ============================================================
    # 6️⃣ Gesamtkosten für Hotelübernachtungen berechnen
    # ============================================================
    # - Basispreis: Preis pro Zimmer * Anzahl Nächte * Anzahl Zimmer
    base_hotel_cost = df['hotel_price_per_room_night_usd'] * df['nights'] * df['rooms']

    # - Wenn Rabatt aktiv ist, Rabatt anwenden
    df['money_spent_hotel'] = np.where(
        df['hotel_discount'],
        base_hotel_cost * (1 - df['hotel_discount_amount']),
        base_hotel_cost
    )

    # ============================================================
    # 7️⃣ Zeitspanne zwischen Buchung und Abflug bestimmen
    # ============================================================
    # - Differenz (in Tagen) zwischen Abflugzeitpunkt und Sitzungsende
    df['time_after_booking'] = (df['departure_time'] - df['session_end']).dt.days

    # ============================================================
    # 8️⃣ Reisekategorien anhand von Bedingungen klassifizieren
    # ============================================================
    # - group_trip: mehrere Sitzplätze und mehrere Zimmer (Gruppenreise)
    # - pair_trip: 2 Sitzplätze, 1 Zimmer (Paarreise)
    # - business_week_trip: werktägige Geschäftsreise
    # - weekend_trip: Wochenendreise
    # - discount_trip: Reise mit Rabatt
    # - season: Jahreszeit der Abreise
    #df['group_trip'] = df.apply(is_group_trip, axis=1)
    #df['pair_trip'] = df.apply(is_pair_trip, axis=1)
    #df['business_week_trip'] = df.apply(is_business_week_trip, axis=1)
    #df['weekend_trip'] = df.apply(is_weekend_trip_new, axis=1)
    #df['discount_trip'] = df.apply(is_discount_trip_new, axis=1)
    df['season'] = df.apply(get_season, axis=1)

    # ============================================================
    # 9️⃣ One-Hot-Encoding der Jahreszeit (season)
    # ============================================================
    # - Wandelt die saisonale Kategorie in binäre Spalten um (z. B. season_summer, season_winter)
    encoder = OneHotEncoder(sparse_output=False)
    season_encoded = encoder.fit_transform(df[['season']])
    season_cols = encoder.get_feature_names_out(['season'])
    df[season_cols] = pd.DataFrame(season_encoded, index=df.index)

    # - Originalspalte 'season' entfernen, da sie nun codiert ist
    df.drop(columns='season', inplace=True)

    # ============================================================
    # 🔟 Entfernung zwischen Start- und Zielort berechnen
    # ============================================================
    # - Nutzt die Haversine-Formel, um Distanz (in km) zwischen Flughäfen zu berechnen
    df['distance_km'] = df.apply(
        lambda row: haversine(
            row['home_airport_lat'], row['home_airport_lon'],
            row['destination_airport_lat'], row['destination_airport_lon']
        ),
        axis=1
    )

    # ============================================================
    # 🔚 Rückgabe: bereinigter und angereicherter DataFrame
    # ============================================================
    return df


In [43]:
filter_non_cancelled_trips()

,session_id,user_id,trip_id,session_start,session_end,page_clicks,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,...,num_hotels,money_spent_per_flight,money_spent_per_seat,money_spent_hotel,time_after_booking,season_fall,season_spring,season_summer,season_winter,distance_km
0,542012-004deaac3341435f95193d20c1db24e1,542012,542012-e368760a3dd34190ac40531d5383ec19,2023-02-22 06:22:00,2023-02-22 06:27:42,46,True,0.05,False,NaN,...,1,279.8795,279.879500,419.0,10.0,0.0,1.0,0.0,0.0,1627.049959
1,600078-cd52591b4dcd4dd89779db6b7d1a5042,600078,600078-94092124e6b64001808a3a089ecc4fb0,2023-02-23 09:43:00,2023-02-23 09:44:39,13,False,NaN,False,NaN,...,0,183.0100,183.010000,NaN,2.0,0.0,0.0,0.0,1.0,1000.385118
2,283325-eed61ca33de3458a9134d3d743f17722,283325,283325-dd763c2d6d334b3790266b500393438e,2023-02-28 01:28:00,2023-02-28 01:32:07,33,True,0.10,False,NaN,...,1,407.9430,407.943000,2108.0,5.0,0.0,1.0,0.0,0.0,2650.425613
3,530801-3f8db46473804f22a1bed8f249cc07e0,530801,530801-c1984101944049c3981bca433393ae12,2023-04-19 15:57:00,2023-04-19 16:00:06,25,False,NaN,False,NaN,...,1,620.7100,620.710000,582.0,5.0,0.0,1.0,0.0,0.0,3516.533724
4,570279-7d86b66f8f0b468c90d9553291a4a281,570279,570279-c48e05a90f9c44e8beb667ba9cc1f8a0,2023-05-19 19:32:00,2023-05-19 19:34:50,23,False,NaN,False,NaN,...,1,91.2900,91.290000,468.0,6.0,0.0,1.0,0.0,0.0,539.970431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16094,532823-969284ec22dc4fa4b42c1f694b322265,532823,532823-168ac0568a884d4a8850527b82098cb0,2023-01-19 06:39:00,2023-01-19 06:41:00,16,False,NaN,False,NaN,...,1,388.8200,388.820000,0.0,5.0,0.0,0.0,0.0,1.0,2315.492556
16095,472966-a27a7987341e4963ad9f578073298080,472966,472966-778844a7de0d46bba12ce251406b403f,2023-02-01 12:27:00,2023-02-01 12:30:27,28,False,NaN,False,NaN,...,1,644.9700,644.970000,0.0,5.0,0.0,0.0,0.0,1.0,4135.143760
16096,568970-05c8d154693a40b795bd5d9891c42c4f,568970,568970-d1e479770f9e42b590cb708834c80a96,2023-02-02 20:46:00,2023-02-02 20:47:25,11,False,NaN,True,0.1,...,0,333.1600,333.160000,NaN,9.0,0.0,0.0,0.0,1.0,1713.030005
16097,542277-fb2204a5ef20426b8fe0ef4137e023d9,542277,542277-fbf16e584b494edb859cefc891b07b54,2023-02-06 13:34:00,2023-02-06 13:37:05,25,True,0.05,False,NaN,...,1,563.5400,563.540000,274.0,7.0,0.0,0.0,0.0,1.0,3365.298015


In [44]:
def aggregate_user_trips():
    # ============================================================
    # 1️⃣ Gruppierung nach Nutzer (user_id)
    # ============================================================
    # Ziel: Alle Reisen pro Nutzer aggregieren, um ein Nutzerprofil zu erstellen
    return nc_sessions.groupby('user_id').agg(

        # ========================================================
        # 🔹 Eindeutige Zählungen (Unique Counts)
        # ========================================================
        # Anzahl unterschiedlicher Reisen pro Nutzer
        num_trips=('trip_id', 'nunique'),
        # Anzahl unterschiedlicher Reiseziele
        num_destinations=('destination', 'nunique'),

        # ========================================================
        # 🔹 Summen (Gesamtwerte über alle Reisen)
        # ========================================================
        # Gesamtanzahl gebuchter Flüge
        num_flights=('num_flights', 'sum'),
        # Gesamtanzahl gebuchter Hotels
        num_hotels=('num_hotels', 'sum'),
        # Anzahl an Gruppenreisen
        #num_group_trips=('group_trip', 'sum'),
        # Anzahl an Paarreisen
        #num_pair_trips=('pair_trip', 'sum'),
        # Anzahl an Geschäftsreisen (unter der Woche)
        #num_business_week_trips=('business_week_trip', 'sum'),
        # Anzahl an Wochenendreisen
        #num_weekend_trips=('weekend_trip', 'sum'),
        # Anzahl an Reisen mit Rabatt
        #num_discount_trips=('discount_trip', 'sum'),
        # Gesamtausgaben für Hotels
        #money_spent_hotel=('money_spent_hotel', 'sum'),
        # Gesamtausgaben für Flüge
        #money_spent_flight=('money_spent_per_flight', 'sum'),
        # Anzahl der Reisen pro Jahreszeit
        #num_summer_trips=('season_summer', 'sum'),
        #num_winter_trips=('season_winter', 'sum'),
        #num_fall_trips=('season_fall', 'sum'),
        #num_spring_trips=('season_spring', 'sum'),

        # ========================================================
        # 🔹 Durchschnittswerte (Mittelwerte über alle Reisen)
        # ========================================================
        # Durchschnittlich ausgegebener Betrag pro Flug
        avg_money_spent_flight=('money_spent_per_flight', 'mean'),
        # Durchschnittliche Zeit zwischen Buchung und Abflug (in Tagen)
        avg_time_after_booking=('time_after_booking', 'mean'),
        # Durchschnittlicher Flugpreis pro Sitzplatz
        avg_money_spent_per_seat=('money_spent_per_seat', 'mean'),
        # Durchschnittliche Hotelkosten pro Reise
        avg_money_spent_hotel=('money_spent_hotel', 'mean'),
        # Durchschnittliche Flugstrecke in Kilometern
        avg_km_flown=('distance_km', 'mean'),
        # Durchschnittliche Anzahl aufgegebener Gepäckstücke pro Reise
        avg_bags=('checked_bags', 'mean')
    ).reset_index()  # Index zurücksetzen, um user_id wieder als Spalte zu haben


In [45]:
user_base_2 = aggregate_user_trips()

In [46]:
def finalize_user_table():
    # ============================================================
    # 1️⃣ Geburtsdatum konvertieren und Alter berechnen
    # ============================================================
    # - Wandelt die Spalte 'birthdate' in ein echtes Datumsformat um
    # - Berechnet das Alter der Nutzer in Jahren (heutiges Datum - Geburtsdatum)
    users['birthdate'] = pd.to_datetime(users['birthdate'], format='mixed')
    users['age'] = (pd.Timestamp.today() - users['birthdate']).dt.days / 365

    # ============================================================
    # 2️⃣ Nutzermerkmale aus verschiedenen Quellen zusammenführen
    # ============================================================
    # - Verknüpft user_base und user_base_2 über die Spalte 'user_id'
    # - Fügt anschließend die demografischen Nutzerdaten ('users') hinzu
    df_user_base = pd.merge(user_base, user_base_2, on='user_id', how='left')
    df_user_base = pd.merge(df_user_base, users, on='user_id', how='left')

    # ============================================================
    # 3️⃣ Entfernen nicht benötigter oder redundanter Spalten
    # ============================================================
    # - Entfernt Basisinformationen, die für Analysen oder Modellierung nicht nötig sind
    drop_cols = ['birthdate', 'home_airport', 'home_airport_lat', 'home_airport_lon', 'sign_up_date']
    df_user_base.drop(columns=drop_cols, inplace=True, errors='ignore')
    
    # Gesamtbetrag, den der Nutzer ausgegeben hat (Hotel + Flug)
    #df_user_base['total_spend'] = (
    #df_user_base['money_spent_hotel'] + df_user_base['money_spent_flight']
    #)

    # Stornorate: Anteil der stornierten Reisen an allen gebuchten Reisen
    # Falls num_trips = 0 → 0, um Division durch 0 zu vermeiden
    #df_user_base['cancellation_rate'] = np.where(
    #df_user_base['num_trips'] > 0,
    #df_user_base['num_canceled_trips'] / df_user_base['num_trips'],
    0
    #)

    # Browsing Rate: Verhältnis leerer Sessions zu allen Sessions
    #df_user_base['browsing_rate'] = (
    #df_user_base['num_empty_sessions'] / df_user_base['num_sessions']
    #)

    # Anteil der Geschäftsreisen an allen Reisen
    #df_user_base['business_rate'] = np.where(
    #df_user_base['num_trips'] > 0,
    #df_user_base['num_business_week_trips'] / df_user_base['num_trips'],
    0
  #)

    # Anteil der Gruppenreisen an allen Reisen
    #df_user_base['group_rate'] = np.where(
    #df_user_base['num_trips'] > 0,
    #df_user_base['num_group_trips'] / df_user_base['num_trips'],
    0
 #)


    # ============================================================
    # 4️⃣ Bereinigte und angereicherte Nutzertabelle speichern
    # ============================================================
    # - Speichert den finalen Nutzer-Datensatz im definierten Feature-Metrics-Verzeichnis
    save_path = os.path.join(feature_metrics_path, "user_base.csv")
    df_user_base.to_csv(save_path, index=False)

    # ============================================================
    # 5️⃣ Rückgabe: finale Nutzertabelle mit allen Features
    # ============================================================
    return df_user_base


In [47]:
finalize_user_table()

,user_id,num_clicks,avg_session_clicks,num_empty_sessions,num_canceled_trips,num_sessions,avg_session_duration,num_trips,num_destinations,num_flights,...,avg_money_spent_per_seat,avg_money_spent_hotel,avg_km_flown,avg_bags,gender,married,has_children,home_country,home_city,age
0,23557,82,10.250,6,0,8,76.625,2.0,0.0,0.0,...,NaN,1835.250000,NaN,NaN,F,True,False,usa,new york,67.000000
1,94883,73,9.125,6,0,8,67.750,2.0,2.0,4.0,...,276.252500,65.000000,1453.666128,0.500000,F,True,False,usa,kansas city,53.720548
2,101486,131,16.375,6,0,8,122.250,2.0,1.0,2.0,...,189.910000,1099.500000,965.340568,0.000000,F,True,True,usa,tacoma,52.991781
3,101961,126,15.750,3,0,8,117.750,5.0,5.0,10.0,...,247.538600,485.800000,1321.684183,0.400000,F,True,False,usa,boston,45.216438
4,106907,240,30.000,6,0,8,201.250,1.0,1.0,2.0,...,2317.010000,4257.000000,13402.323077,5.000000,F,True,True,usa,miami,47.043836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,792549,114,14.250,4,0,8,106.875,4.0,4.0,8.0,...,259.792500,144.000000,1437.965220,0.500000,F,False,False,usa,kansas city,47.854795
5994,796032,148,18.500,4,0,8,143.875,3.0,3.0,6.0,...,578.015667,630.500000,3510.032613,1.000000,F,True,False,canada,winnipeg,52.934247
5995,801660,115,14.375,5,0,8,106.000,3.0,1.0,6.0,...,129.346833,290.666667,721.127119,0.333333,F,True,True,canada,toronto,56.090411
5996,811077,105,13.125,7,0,8,99.125,1.0,1.0,2.0,...,579.790000,852.000000,3184.843425,0.000000,F,True,True,usa,knoxville,46.778082
